In [40]:
import pandas as pd
import numpy as np

In [41]:
#Read the installments_payments.csv,POS_CASH_balance.csv, credit_card_balance.csv, previous_application.csv

installments_payments=pd.read_csv('installments_payments.csv')
POS_CASH_balance=pd.read_csv('POS_CASH_balance.csv')
credit_card_balance=pd.read_csv('credit_card_balance.csv')
#previous_app=pd.read_csv('previous_application.csv')

## installments_payments **Feature Engineering** ve **Aggregation** 

In [42]:
ins=installments_payments.copy()

In [43]:
ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
SK_ID_PREV                int64
SK_ID_CURR                int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER     int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


Aggregation öncesi **Feature Engineering**

In [44]:
'''gec odemeye odaklandik. Iki degisken türettik. Odemenin odendigi gunun odeme tarihinden 
kac gün önce sonra olma durumu. Pozitif sonuc erken ödenen gün sayisi negatif sonuc gec ödenenen gün
sayisi degeridir.'''

ins['NEW_DAYS_PAID_EARLIER']=ins['DAYS_INSTALMENT']-ins['DAYS_ENTRY_PAYMENT']

# Her bir taksit ödemesinin gec olup olmama durumu 1: gec ödedi 0: erken ödemeyi temsil eder

ins['NEW_NUM_PAID_LATER']=ins['NEW_DAYS_PAID_EARLIER'].map(lambda x:1 if x<0 else 0)

**Aggregation Islemi**

In [45]:
# Agrregation ve degisken tekillestirme

ins_agg=ins.groupby('SK_ID_PREV').agg({'NUM_INSTALMENT_VERSION':['nunique'],
                                      'NUM_INSTALMENT_NUMBER':'max',
                                      'DAYS_INSTALMENT':['min','max'],
                                      'DAYS_ENTRY_PAYMENT':['min','max'],
                                      'AMT_INSTALMENT':['min','max','sum','mean'],
                                      'AMT_PAYMENT':['min','max','sum','mean'],
                                      'NEW_DAYS_PAID_EARLIER':'mean',
                                      'NEW_NUM_PAID_LATER':'sum'})

In [46]:
ins_agg.head()

NUM_INSTALMENT_VERSION NUM_INSTALMENT_NUMBER DAYS_INSTALMENT  \
                          nunique                   max             min   
SK_ID_PREV                                                                
1000001                         2                     2          -268.0   
1000002                         2                     4         -1600.0   
1000003                         1                     3           -94.0   
1000004                         2                     7          -862.0   
1000005                         1                    10         -1688.0   

                   DAYS_ENTRY_PAYMENT         AMT_INSTALMENT             \
               max                min     max            min        max   
SK_ID_PREV                                                                
1000001     -238.0             -294.0  -244.0        6404.31  62039.115   
1000002    -1510.0            -1611.0 -1554.0        6264.00  18443.565   
1000003      -34.0             -108.0   -49.0        4951.35   4951.350   
1000004     -682.0             -881.0  -695.0        3391.11  13176.495   
1000005    -1418.0            -1687.0 -1433.0       14599.26  14713.605   

                                     AMT_PAYMENT                         \
                   sum          mean         min        max         sum   
SK_ID_PREV                                                                
1000001      68443.425  34221.712500     6404.31  62039.115   68443.425   
1000002      37235.565   9308.891250     6264.00  18443.565   37235.565   
1000003      14854.050   4951.350000     4951.35   4951.350   14854.050   
1000004      33523.155   4789.022143     3391.11  13176.495   33523.155   
1000005     161735.310  14703.210000        2.79  14713.605  147021.705   

                         NEW_DAYS_PAID_EARLIER NEW_NUM_PAID_LATER  
                    mean                  mean                sum  
SK_ID_PREV                                                         
1000001     34221.712500             16.000000                  0  
1000002      9308.891250             19.750000                  0  
1000003      4951.350000             15.333333                  0  
1000004      4789.022143             26.714286                  0  
1000005     13365.609545              8.454545                  2

In [47]:
# Multi index problemi cözümü

ins_agg.columns=pd.Index([ind[0]+'_'+ind[1].upper() for ind in ins_agg.columns.tolist()])

In [48]:
ins_agg.head()

,NUM_INSTALMENT_VERSION_NUNIQUE,NUM_INSTALMENT_NUMBER_MAX,DAYS_INSTALMENT_MIN,DAYS_INSTALMENT_MAX,DAYS_ENTRY_PAYMENT_MIN,DAYS_ENTRY_PAYMENT_MAX,AMT_INSTALMENT_MIN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MIN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_MEAN,NEW_DAYS_PAID_EARLIER_MEAN,NEW_NUM_PAID_LATER_SUM
SK_ID_PREV,,,,,,,,,,,,,,,,
1000001,2,2,-268.0,-238.0,-294.0,-244.0,6404.31,62039.115,68443.425,34221.712500,6404.31,62039.115,68443.425,34221.712500,16.000000,0
1000002,2,4,-1600.0,-1510.0,-1611.0,-1554.0,6264.00,18443.565,37235.565,9308.891250,6264.00,18443.565,37235.565,9308.891250,19.750000,0
1000003,1,3,-94.0,-34.0,-108.0,-49.0,4951.35,4951.350,14854.050,4951.350000,4951.35,4951.350,14854.050,4951.350000,15.333333,0
1000004,2,7,-862.0,-682.0,-881.0,-695.0,3391.11,13176.495,33523.155,4789.022143,3391.11,13176.495,33523.155,4789.022143,26.714286,0
1000005,1,10,-1688.0,-1418.0,-1687.0,-1433.0,14599.26,14713.605,161735.310,14703.210000,2.79,14713.605,147021.705,13365.609545,8.454545,2


Aggregation sonrasi **Feature Engineering**

In [49]:
# bazi degiskenleri drop ettik

ins_agg.drop(['DAYS_INSTALMENT_MIN',
               'DAYS_INSTALMENT_MAX',
               'DAYS_ENTRY_PAYMENT_MIN',
               'DAYS_ENTRY_PAYMENT_MAX'],axis=1,inplace=True)

In [50]:
# Kredi ödeme yüzdesi ve toplam kalan borc

ins_agg['NEW_PAYMENT_PERC'] = ins_agg['AMT_PAYMENT_SUM'] / ins_agg['AMT_INSTALMENT_SUM']
ins_agg['NEW_PAYMENT_DIFF'] = ins_agg['AMT_INSTALMENT_SUM'] - ins_agg['AMT_PAYMENT_SUM']

In [51]:
ins_agg.head()

,NUM_INSTALMENT_VERSION_NUNIQUE,NUM_INSTALMENT_NUMBER_MAX,AMT_INSTALMENT_MIN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MIN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_MEAN,NEW_DAYS_PAID_EARLIER_MEAN,NEW_NUM_PAID_LATER_SUM,NEW_PAYMENT_PERC,NEW_PAYMENT_DIFF
SK_ID_PREV,,,,,,,,,,,,,,
1000001,2,2,6404.31,62039.115,68443.425,34221.712500,6404.31,62039.115,68443.425,34221.712500,16.000000,0,1.000000,0.000
1000002,2,4,6264.00,18443.565,37235.565,9308.891250,6264.00,18443.565,37235.565,9308.891250,19.750000,0,1.000000,0.000
1000003,1,3,4951.35,4951.350,14854.050,4951.350000,4951.35,4951.350,14854.050,4951.350000,15.333333,0,1.000000,0.000
1000004,2,7,3391.11,13176.495,33523.155,4789.022143,3391.11,13176.495,33523.155,4789.022143,26.714286,0,1.000000,0.000
1000005,1,10,14599.26,14713.605,161735.310,14703.210000,2.79,14713.605,147021.705,13365.609545,8.454545,2,0.909027,14713.605


## POS_CASH_balance **Feature Engineering** ve **Aggregation**

In [52]:
pos=POS_CASH_balance.copy()

In [53]:
# Kategorik Degiskenimizi Dummy Degiskenine Dönüstürme

pos=pd.get_dummies(pos,columns=['NAME_CONTRACT_STATUS'],dummy_na=True)

In [54]:
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0


In [55]:
# Duplicate kontrolu icin dongu

a_list=[]
for j in pos["SK_ID_PREV"]:
    a_list.append(j)  
# a_set = set(a_list)
# contains_duplicates = len(a_list) != len(a_set)

contains_duplicates = any(a_list.count(element) > 1 for element in a_list)

print(contains_duplicates)

True


In [56]:
# Aggregation Islemi - Tekillestirme

pos_agg=pos.groupby('SK_ID_PREV').agg({'MONTHS_BALANCE':['min','max'],
                                        'CNT_INSTALMENT':['min','max'],
                                        'CNT_INSTALMENT_FUTURE':['min','max'],
                                        'SK_DPD':['max','mean'],
                                        'SK_DPD_DEF':['max','mean'],
                                        'NAME_CONTRACT_STATUS_Active':'sum',
                                        'NAME_CONTRACT_STATUS_Amortized debt':'sum',
                                        'NAME_CONTRACT_STATUS_Approved':'sum',
                                        'NAME_CONTRACT_STATUS_Canceled':'sum',
                                        'NAME_CONTRACT_STATUS_Completed':'sum',
                                        'NAME_CONTRACT_STATUS_Demand':'sum',
                                        'NAME_CONTRACT_STATUS_Returned to the store':'sum',
                                        'NAME_CONTRACT_STATUS_Signed':'sum',
                                        'NAME_CONTRACT_STATUS_XNA':'sum',
                                        'NAME_CONTRACT_STATUS_nan':'sum'
                                      })

In [57]:
pos_agg.head()

MONTHS_BALANCE     CNT_INSTALMENT       CNT_INSTALMENT_FUTURE  \
                      min max            min   max                   min   
SK_ID_PREV                                                                 
1000001               -10  -8            2.0  12.0                   0.0   
1000002               -54 -50            4.0   6.0                   0.0   
1000003                -4  -1           12.0  12.0                   9.0   
1000004               -29 -22            7.0  10.0                   0.0   
1000005               -56 -46           10.0  10.0                   0.0   

                 SK_DPD      SK_DPD_DEF      NAME_CONTRACT_STATUS_Active  \
             max    max mean        max mean                         sum   
SK_ID_PREV                                                                 
1000001     12.0      0  0.0          0  0.0                           2   
1000002      4.0      0  0.0          0  0.0                           4   
1000003     12.0      0  0.0          0  0.0                           4   
1000004     10.0      0  0.0          0  0.0                           7   
1000005     10.0      0  0.0          0  0.0                          10   

           NAME_CONTRACT_STATUS_Amortized debt NAME_CONTRACT_STATUS_Approved  \
                                           sum                           sum   
SK_ID_PREV                                                                     
1000001                                      0                             0   
1000002                                      0                             0   
1000003                                      0                             0   
1000004                                      0                             0   
1000005                                      0                             0   

           NAME_CONTRACT_STATUS_Canceled NAME_CONTRACT_STATUS_Completed  \
                                     sum                            sum   
SK_ID_PREV                                                                
1000001                                0                              1   
1000002                                0                              1   
1000003                                0                              0   
1000004                                0                              1   
1000005                                0                              1   

           NAME_CONTRACT_STATUS_Demand  \
                                   sum   
SK_ID_PREV                               
1000001                              0   
1000002                              0   
1000003                              0   
1000004                              0   
1000005                              0   

           NAME_CONTRACT_STATUS_Returned to the store  \
                                                  sum   
SK_ID_PREV                                              
1000001                                             0   
1000002                                             0   
1000003                                             0   
1000004                                             0   
1000005                                             0   

           NAME_CONTRACT_STATUS_Signed NAME_CONTRACT_STATUS_XNA  \
                                   sum                      sum   
SK_ID_PREV                                                        
1000001                              0                        0   
1000002                              0                        0   
1000003                              0                        0   
1000004                              0                        0   
1000005                              0                        0   

           NAME_CONTRACT_STATUS_nan  
                                sum  
SK_ID_PREV                           
1000001                           0  
1000002                           0  
1000003                       

In [58]:
# Multilayer index'i tek boyutlu index'e dönüstürme

pos_agg.columns=pd.Index([ind[0]+'_'+ind[1].upper() for ind in pos_agg.columns.tolist()])

In [59]:
pos_agg.head()

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_FUTURE_MIN,CNT_INSTALMENT_FUTURE_MAX,SK_DPD_MAX,SK_DPD_MEAN,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,NAME_CONTRACT_STATUS_Active_SUM,NAME_CONTRACT_STATUS_Amortized debt_SUM,NAME_CONTRACT_STATUS_Approved_SUM,NAME_CONTRACT_STATUS_Canceled_SUM,NAME_CONTRACT_STATUS_Completed_SUM,NAME_CONTRACT_STATUS_Demand_SUM,NAME_CONTRACT_STATUS_Returned to the store_SUM,NAME_CONTRACT_STATUS_Signed_SUM,NAME_CONTRACT_STATUS_XNA_SUM,NAME_CONTRACT_STATUS_nan_SUM
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,
1000001,-10,-8,2.0,12.0,0.0,12.0,0,0.0,0,0.0,2,0,0,0,1,0,0,0,0,0
1000002,-54,-50,4.0,6.0,0.0,4.0,0,0.0,0,0.0,4,0,0,0,1,0,0,0,0,0
1000003,-4,-1,12.0,12.0,9.0,12.0,0,0.0,0,0.0,4,0,0,0,0,0,0,0,0,0
1000004,-29,-22,7.0,10.0,0.0,10.0,0,0.0,0,0.0,7,0,0,0,1,0,0,0,0,0
1000005,-56,-46,10.0,10.0,0.0,10.0,0,0.0,0,0.0,10,0,0,0,1,0,0,0,0,0


Aggregation sonrasi **Feature Engineering**

In [60]:
# SK_DPD kac kredide 0 olma durumu (SK_DPD MAX alacagiz 0 durumunu veriyor) 
# SK_DPD_DEF (SK_DPD_DEF_MAX sifir olma durumunu veriyor)
# CNT_INSTALMENT_FUTURE_MIN==0 oldugunda NAME_CONTRACT_STATUS_Completed_SUM==0 olma durumu 

pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']= (pos_agg['CNT_INSTALMENT_FUTURE_MIN']==0) & (pos_agg['NAME_CONTRACT_STATUS_Completed_SUM']==0)


# 1:kredi zamaninda kapanmamis 
# 0:kredi zamaninda kapanmis

pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']=pos_agg['NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME'].astype(int)

In [61]:
pos_agg.drop(['NAME_CONTRACT_STATUS_Approved_SUM',
               'NAME_CONTRACT_STATUS_Amortized debt_SUM',
               'NAME_CONTRACT_STATUS_Canceled_SUM',
               'NAME_CONTRACT_STATUS_Returned to the store_SUM',
               'NAME_CONTRACT_STATUS_Signed_SUM',
               'NAME_CONTRACT_STATUS_XNA_SUM',
               'NAME_CONTRACT_STATUS_nan_SUM'],axis=1,inplace=True)

In [62]:
pos_agg.head()

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_MAX,CNT_INSTALMENT_FUTURE_MIN,CNT_INSTALMENT_FUTURE_MAX,SK_DPD_MAX,SK_DPD_MEAN,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,NAME_CONTRACT_STATUS_Active_SUM,NAME_CONTRACT_STATUS_Completed_SUM,NAME_CONTRACT_STATUS_Demand_SUM,NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME
SK_ID_PREV,,,,,,,,,,,,,,
1000001,-10,-8,2.0,12.0,0.0,12.0,0,0.0,0,0.0,2,1,0,0
1000002,-54,-50,4.0,6.0,0.0,4.0,0,0.0,0,0.0,4,1,0,0
1000003,-4,-1,12.0,12.0,9.0,12.0,0,0.0,0,0.0,4,0,0,0
1000004,-29,-22,7.0,10.0,0.0,10.0,0,0.0,0,0.0,7,1,0,0
1000005,-56,-46,10.0,10.0,0.0,10.0,0,0.0,0,0.0,10,1,0,0


In [63]:
pos_agg.columns

Index(['MONTHS_BALANCE_MIN', 'MONTHS_BALANCE_MAX', 'CNT_INSTALMENT_MIN',
       'CNT_INSTALMENT_MAX', 'CNT_INSTALMENT_FUTURE_MIN',
       'CNT_INSTALMENT_FUTURE_MAX', 'SK_DPD_MAX', 'SK_DPD_MEAN',
       'SK_DPD_DEF_MAX', 'SK_DPD_DEF_MEAN', 'NAME_CONTRACT_STATUS_Active_SUM',
       'NAME_CONTRACT_STATUS_Completed_SUM', 'NAME_CONTRACT_STATUS_Demand_SUM',
       'NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME'],
      dtype='object')

## credit_card_balance **Feature Engineering** ve **Aggregation**

In [64]:
CCB = credit_card_balance.copy()

In [65]:
# bununla beraber artik tumu sayisal degerler

CCB = pd.get_dummies(CCB, columns= ['NAME_CONTRACT_STATUS'] )  

In [66]:
dropthis = ['NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Demand',
           'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Sent proposal',
           'NAME_CONTRACT_STATUS_Signed' ]

CCB = CCB.drop(dropthis, axis=1)

In [67]:
CCB = pd.get_dummies(CCB, dummy_na= True)

In [68]:
CCB.drop(['SK_ID_PREV'], axis= 1, inplace = True)

In [69]:
CCB_agg = CCB.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum'])

In [70]:
CCB_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in CCB_agg.columns.tolist()])

In [72]:
CCB_agg['CC_COUNT'] = CCB.groupby('SK_ID_CURR').size()

In [73]:
CCB_agg.head()

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_CREDIT_LIMIT_ACTUAL_MIN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,...,CC_SK_DPD_DEF_SUM,CC_NAME_CONTRACT_STATUS_Active_MIN,CC_NAME_CONTRACT_STATUS_Active_MAX,CC_NAME_CONTRACT_STATUS_Active_MEAN,CC_NAME_CONTRACT_STATUS_Active_SUM,CC_NAME_CONTRACT_STATUS_Completed_MIN,CC_NAME_CONTRACT_STATUS_Completed_MAX,CC_NAME_CONTRACT_STATUS_Completed_MEAN,CC_NAME_CONTRACT_STATUS_Completed_SUM,CC_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,0.0,0.00,0.000000,0.000,270000,270000,...,0,1,1,1.000000,6,0,0,0.000000,0,6
100011,-75,-2,-38.5,-2849,0.0,189000.00,54482.111149,4031676.225,90000,180000,...,0,1,1,1.000000,74,0,0,0.000000,0,74
100013,-96,-1,-48.5,-4656,0.0,161420.22,18159.919219,1743352.245,45000,157500,...,1,1,1,1.000000,96,0,0,0.000000,0,96
100021,-18,-2,-10.0,-170,0.0,0.00,0.000000,0.000,675000,675000,...,0,0,1,0.411765,7,0,1,0.588235,10,17
100023,-11,-4,-7.5,-60,0.0,0.00,0.000000,0.000,45000,225000,...,0,1,1,1.000000,8,0,0,0.000000,0,8


## **Diger tablolarla Birlestirme (JOIN Islemi)**

In [74]:
df=pos_agg.join(ins_agg,
                how='left',
                on='SK_ID_PREV')

In [75]:
df=df.join(CCB_agg,
          how='left',
          on='SK_ID_PREV')

In [76]:
(df.shape,CCB_agg.shape,ins_agg.shape,pos_agg.shape)

((936325, 117), (103558, 89), (997752, 14), (936325, 14))